# APIM ❤️ OpenAI

## AI Foundry SDK lab
![flow](../../images/ai-foundry-sdk.gif)

This experimentation involves integrating [Azure AI Foundry SDK](https://learn.microsoft.com/azure/ai-studio/how-to/develop/sdk-overview?tabs=async&pivots=programming-language-python) with APIM. The OpenAI connection in the AI Foundry project includes an APIM endpoint and subscription, allowing client application requests to seamlessly route through APIM when utilizing the AI Foundry SDK.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create deployment using 🦾 Bicep](#1)
- [2️⃣ Get the deployment outputs](#2)
- [3️⃣ Install Python packages](#3)
- [🧪 Test a chat completion](#requests)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "eastus"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_deployment_name = "gpt-4o-mini"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ ]

log_analytics_name = "workspace"
app_insights_name = 'insights'

ai_studio_hub_name = 'hub'
ai_studio_project_name = 'project'
storage_account_name = 'storage'
keyvault_name = 'keyvault'
container_registry_name = 'registry'

openai_embeddings_deployment_name = "text-embedding-ada-002"
openai_embeddings_model_name = "text-embedding-ada-002"
openai_embeddings_model_version = "2"
searchservice_resource_name = "search"
searchservice_sku = "standard"
searchindex_name = "example-index"





<a id='1'></a>
### 1️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())
    
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "aiStudioHubName": { "value": ai_studio_hub_name },
    "aiStudioProjectName": { "value": ai_studio_project_name },
    "storageAccountName": { "value": storage_account_name }, 
    "keyVaultName": { "value": keyvault_name },
    "containerRegistryName": { "value": container_registry_name },
    "openAIEmbeddingsDeploymentName": { "value": openai_embeddings_deployment_name},
    "openAIEmbeddingsModelName": { "value": openai_embeddings_model_name},
    "openAIEmbeddingsModelVersion": { "value": openai_embeddings_model_version},
    "searchServiceName": { "value": searchservice_resource_name},
    "searchServiceSku": { "value": searchservice_sku}
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


<a id='2'></a>
### 2️⃣ Get the deployment outputs


In [ ]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.projectConnectionString.value -o tsv
project_connection_string = deployment_stdout.n
print("👉🏻 Project Connection String: ", project_connection_string)



<a id='3'></a>
### 3️⃣ Install Python packages

Update [requirements.txt](requirements.txt) according the [documentation](https://learn.microsoft.com/azure/ai-studio/tutorials/copilot-sdk-create-resources?tabs=windows#install-packages).

In [ ]:
! pip install -r requirements.txt

<a id='requests'></a>
### 🧪 Test a chat completion

The Bicep deployment incorporated an OpenAI service connection into the AI Foundry project using the APIM endpoint and subscription. The following code retrieves the AI Foundry project, obtains the Azure OpenAI connection with the APIM endpoint, and then tests a simple chat completion.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient


project = AIProjectClient.from_connection_string(
  conn_str=project_connection_string,
  credential=DefaultAzureCredential())

messages=[
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

openai = project.inference.get_azure_openai_client(api_version="2024-06-01")

response = openai.chat.completions.create(
    model=openai_deployment_name,
    messages=messages
)

print("💬 ", response.choices[0].message.content)




<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.